In [ ]:
import os
import numpy as np
import xarray as xr
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LinearSegmentedColormap, Normalize

# Load datasets for wind direction and speed
dir_data = xr.open_dataset('../../data/th.nc')
speed_data = xr.open_dataset('../../data/vs.nc')

# Define a custom color map with different shades of blue for visualization
colors = ["#e0f7fa", "#b2ebf2", "#80deea", "#4dd0e1", "#26c6da",
          "#00bcd4", "#00acc1", "#0097a7", "#00838f", "#006064",
          "#004f5c", "#003a4a", "#002938", "#001921", "#000c12"]
custom_cmap = LinearSegmentedColormap.from_list("custom_blue_15", colors, N=15)

# Define a downsampling factor to manage resolution for quiver plots
downsample_factor = 10

# List of days to visualize (selected days covering specific storms)
day_list = [32,33,34,35,36,87,88,89,90,91]
num_frames = len(day_list)

# Normalize wind speed values to a range for color mapping in plots
norm = Normalize(vmin=0, vmax=10)

print("Running...")

# Main function to generate streamlines images for each selected day
def generate_images():
    os.makedirs('../images/streamlines', exist_ok=True)  # Ensure output directory exists

    for day in day_list:
        # Initialize plot for each day
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Create a Basemap projection for the region (USA and surroundings)
        m = Basemap(llcrnrlon=-123, llcrnrlat=20, urcrnrlon=-62, urcrnrlat=50,
                   projection='lcc', lat_1=33, lat_2=45, lat_0=39.5, lon_0=-98, ax=ax)

        # Define grid resolution based on map boundaries (50km spacing)
        nx = int((m.xmax - m.xmin) / 50000)
        ny = int((m.ymax - m.ymin) / 50000)

        # Generate mesh grid for x and y coordinates in the map's projection
        x = np.linspace(m.xmin, m.xmax, nx)
        y = np.linspace(m.ymin, m.ymax, ny)
        x_grid, y_grid = np.meshgrid(x, y)

        # Convert projection grid to geographic coordinates (lat/lon)
        lon_grid, lat_grid = m(x_grid, y_grid, inverse=True)

        # Extract wind direction and speed for the specific day from datasets
        wind_dir = dir_data['wind_from_direction'].isel(day=day)
        wind_speed = speed_data['wind_speed'].isel(day=day)

        # Convert wind direction to radians and calculate U, V components
        wind_rad = np.deg2rad(270 - wind_dir)  # Convert from meteorological to math convention
        U = wind_speed * np.cos(wind_rad)      # U component of wind (east-west)
        V = wind_speed * np.sin(wind_rad)      # V component of wind (north-south)

        # Get latitude and longitude arrays from dataset for interpolation
        lats = dir_data.lat.values
        lons = dir_data.lon.values

        # Set up interpolators to resample wind data on the map grid
        U_interp = RegularGridInterpolator((lats, lons), U.values,
                                           bounds_error=False, fill_value=0)
        V_interp = RegularGridInterpolator((lats, lons), V.values,
                                           bounds_error=False, fill_value=0)
        speed_interp = RegularGridInterpolator((lats, lons), wind_speed.values,
                                               bounds_error=False, fill_value=0)

        # Interpolate U, V, and speed on the projection grid
        points = np.column_stack((lat_grid.flatten(), lon_grid.flatten()))
        U_grid = U_interp(points).reshape(x_grid.shape)
        V_grid = V_interp(points).reshape(x_grid.shape)
        speed_grid = speed_interp(points).reshape(x_grid.shape)

        # Rotate U and V to align with the map's grid orientation
        U_grid, V_grid = m.rotate_vector(U_grid, V_grid, lon_grid, lat_grid)

        # Draw base map details (boundaries, continents, coastlines)
        m.drawmapboundary(fill_color='#A6CAE0')
        m.fillcontinents(color='#E1DCBD', lake_color='#A6CAE0', alpha=0.7)
        m.drawcoastlines(color='#404040', linewidth=0.8)
        m.drawcountries(color='#404040', linewidth=0.6)
        m.drawparallels(np.arange(20,51,10), labels=[1,0,0,0], fontsize=8, color='#808080', linewidth=0.5)
        m.drawmeridians(np.arange(-120,-60,10), labels=[0,0,0,1], fontsize=8, color='#808080', linewidth=0.5)

        # Plot wind streamlines, colored by wind speed, with density set for clarity
        stream = ax.streamplot(x, y, U_grid, V_grid,
                               color=speed_grid,
                               cmap=custom_cmap,
                               norm=norm,
                               linewidth=1,
                               density=2)

        # Add color bar to indicate wind speed scale
        plt.colorbar(stream.lines, label='Wind Speed (m/s)', orientation='vertical', fraction=0.046, aspect=10)

        # Set title for each frame and adjust layout for clear output
        plt.title(f"Wind Streamlines - Day {day}", fontsize=14, pad=20)
        plt.tight_layout()

        # Save each frame as an individual image in the specified directory
        fig.savefig(f'../images/streamlines/day_{day:02d}.png', format='png')
        plt.close(fig)  # Close the figure to free memory

# Function to compile saved images into a GIF animation
def create_gif():
    images = []
    for day in day_list:
        # Read each saved image and add it to the list for GIF creation
        images.append(imageio.imread(f'../images/streamlines/day_{day:02d}.png'))
    # Save all frames into a single GIF file with a specified frame rate
    imageio.mimsave(f'../gifs/streamlines_{num_frames}.gif', images, fps=2)

# Generate the images and create the GIF
generate_images()
create_gif()
print("Completed")

Running...
Completed
